In [ ]:
# Auto Encoder를 이용해서 특징을 추출하고 랜덤포레스트를 이용하여 값 예측

In [1]:
import pandas as pd
import numpy as np

import copy
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
import torch.nn as nn

from tqdm import tqdm # import tqdm 시 모듈 에러 발생
from sklearn.decomposition import PCA  

# import xgboost as xgb
import lightgbm as lgb

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
tr_input_df= copy.copy(tr_df.drop(['ID','y'],axis=1))
tr_output_df = copy.copy(tr_df['y'])

In [6]:
tr_ae, val_ae = train_test_split(tr_input_df, random_state=42, test_size = 0.05)

In [5]:
tr_ae, val_ae = train_test_split(tr_input_df, random_state=42, test_size = 0.05)

base_input_tr = torch.Tensor(tr_ae.values)
base_input_val = torch.Tensor(val_ae.values)
base_output = torch.Tensor(tr_output_df.values)

In [8]:
ae_dataloader_tr = DataLoader(base_input_tr, batch_size = 8)
ae_dataloader_val = DataLoader(base_input_val, batch_size = 8)

In [97]:
# 보통 3분의 1 또는 4분의 1로 잠재 변수 차원 결정 

class autoencoder(nn.Module):
    def __init__(self): # 직접 은닉층, 잠재 차원 정의 할 거면 여기서! hidden_dim, latent_dim) 선언
        super(autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
                nn.Linear(11, 6),
                nn.ReLU(),
                nn.Linear(6, 3), #  hidden_dim, latent_dim
                nn.ReLU()
        ).to(device)
        
        self.decoder = nn.Sequential(
                nn.Linear(3, 6),  #  latent_dim, hidden_dim
                nn.ReLU(),
                nn.Linear(6, 11),
        ).to(device)
    

        self.optimizer = torch.optim.Adam(
            [
                        {'params' : self.encoder.parameters(), 'lr' : 0.001, 'weight_decay' : 0.01},
                        {'params' : self.decoder.parameters(), 'lr' : 0.001, 'weight_decay' : 0.01}
            ]
        )

        self.criterion = nn.MSELoss()
        
    def forward(self, data):
        encoded = self.encoder(data)
        result_data = self.decoder(encoded)
        
        return result_data
    
    def train_autoencoder(self): # 메서드 명을 train으로 하면 self.train()에서 재귀에러 발생
        self.train()
        patient_limit = 5
        patient_check = 0
        epochs = 30
        best_params = None
        init_val_loss = float('inf')
        for epoch in range(epochs):
            tr_loss = 0
            for tr_batch in tqdm(base_input_tr, total = len(base_input_tr)):
                tr_batch = tr_batch.to(device)
                self.optimizer.zero_grad()
                pred_feature= self.forward(tr_batch)
                loss = self.criterion(pred_feature, tr_batch)
                tr_loss += loss.item()
    
                loss.backward()
                clip_grad_norm_(self.parameters(), max_norm= 1.0)
            
                self.optimizer.step()
            print(f'epoch: {epoch + 1}, 훈련 손실: {(tr_loss / len(base_input_tr)) : 2f}')
            
            self.eval()
            val_loss = 0
            print_loss = 0
            with torch.no_grad():
                for val_batch in base_input_val:
                    val_batch = val_batch.to(device)
                    preds = self.forward(val_batch)
                    loss = self.criterion(preds, val_batch)
                    val_loss += loss.item()
                
                print_loss = val_loss / len(base_input_val)
                print(f'epoch: {epoch + 1}, 검증 손실: {np.round(print_loss,3)}')
                
                if np.round(print_loss,3) < init_val_loss:
                    init_val_loss = np.round(print_loss,3)
                else:
                    patient_check += 1
        
            if patient_check == patient_limit:
                break
       
        weight_path  = 'sams_params/best_param_feature_extraction6.pth'
        torch.save(
            {
             'encoder_state_dict' : self.encoder.state_dict(),
             'decoder_state_dict' : self.decoder.state_dict()
            }, weight_path
             )
            
auto_encoder = autoencoder()

# 'sams_params/best_param_feature_extraction6.pth'  학습률 0.01 ,  epoch: 6, 훈련 손실:  0.003670  , 검증 손실: 0.005

In [98]:
auto_encoder.train_autoencoder()

100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:04<00:00, 695.78it/s]


epoch: 1, 훈련 손실:  0.035655
epoch: 1, 검증 손실: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:04<00:00, 696.82it/s]


epoch: 2, 훈련 손실:  0.003682
epoch: 2, 검증 손실: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:04<00:00, 697.11it/s]


...

epoch: 4, 훈련 손실:  0.003678
epoch: 4, 검증 손실: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:04<00:00, 716.34it/s]


epoch: 5, 훈련 손실:  0.003672
epoch: 5, 검증 손실: 0.005


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:04<00:00, 692.18it/s]

epoch: 6, 훈련 손실:  0.003670
epoch: 6, 검증 손실: 0.005


In [92]:
auto_encoder.train_autoencoder()

100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:03<00:00, 813.10it/s]


epoch: 1, 훈련 손실:  1.095403
epoch: 1, 검증 손실: 1.021


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:03<00:00, 844.75it/s]


epoch: 2, 훈련 손실:  0.993690
epoch: 2, 검증 손실: 0.923


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:03<00:00, 837.30it/s]


...

epoch: 23, 훈련 손실:  0.006851
epoch: 23, 검증 손실: 0.009


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:03<00:00, 827.48it/s]


epoch: 24, 훈련 손실:  0.006824
epoch: 24, 검증 손실: 0.009


100%|█████████████████████████████████████████████████████████████████████████████| 2917/2917 [00:03<00:00, 832.55it/s]

epoch: 25, 훈련 손실:  0.006811
epoch: 25, 검증 손실: 0.01


In [99]:
encoder = nn.Sequential(
                nn.Linear(11, 6),
                nn.ReLU(),
                nn.Linear(6, 3), #  hidden_dim, latent_dim
                nn.ReLU()
        ).to(device)
        
decoder = nn.Sequential(
                nn.Linear(3, 6),  #  latent_dim, hidden_dim
                nn.ReLU(),
                nn.Linear(6, 11),
        ).to(device)

In [100]:
check_point = torch.load('sams_params/best_param_feature_extraction6.pth')  # 가중치가 float 32 타입으로 저장되기 때문에 형 변환 필요

In [101]:
encoder.load_state_dict(check_point['encoder_state_dict'])
decoder.load_state_dict(check_point['decoder_state_dict'])

<All keys matched successfully>

In [102]:
ae_input = torch.tensor(tr_input_df.values, dtype = torch.float32).to(device) # float32 , 텐서 변환

In [103]:
def autoencoder_feature(data):
    encode_res = encoder(data)
    decod_res = decoder(encode_res)

    return decod_res

use_feature = autoencoder_feature(ae_input)  # 오토인코더 사용하여 특징 추출한 결과


use_feature_to_numpy = use_feature.detach().cpu().numpy()
df = pd.DataFrame(use_feature_to_numpy, columns=tr_input_df.columns)

In [64]:
rfr = RandomForestRegressor(random_state=43, bootstrap=True, oob_score = True, n_jobs= -1)

random_params = {
    'n_estimators' : range(10,150,20),
    'max_depth' : [-1, 4,5],
    'min_samples_leaf' :  [10,11,12],
    'max_features' : np.arange(0.6, 1.01, 0.1)
}

n_iter = 100
 
random_rfr = RandomizedSearchCV(rfr, 
                                param_distributions=random_params,
                                n_iter = n_iter,
                                cv = 5,
                                scoring = 'neg_mean_squared_error')

In [10]:
random_rfr.fit(tr_input_df.values, tr_output_df.values)

 # oob 데이터를 검증용 데이터로 사용하기 떄문에 굳이 train_test_split 필요 없음

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, oob_score=True,
                                                   random_state=43),
                   n_iter=100,
                   param_distributions={'max_depth': [-1, 4, 5],
                                        'max_features': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'min_samples_leaf': [10, 11, 12],
                                        'n_estimators': range(10, 150, 20)},
                   scoring='neg_mean_squared_error')

In [21]:
random_rfr_best_model = random_rfr.best_estimator_
random_rfr_best_model.fit(tr_input_df.values, tr_output_df.values)
random_rfr_best_model.oob_score_

In [37]:
from sklearn.inspection import permutation_importance

In [2]:
from sklearn.inspection import permutation_importance

importance_res = permutation_importance(random_rfr_best_model,
                                        tr_input_df.values,
                                        tr_output_df.values,
                                        n_repeats= 10,
                                        scoring='neg_mean_squared_error',
                                        random_state=1)

# 특성의 중요도를 파악하는 방법으로, 예측에 사용된 변수의 중요도를 확인할 수 있음

importance_res['importances_mean'].reshape(1,-1)  # 중요도 가장 적은 변수 하나씩 제거해서 oob_score 좋아질 때 까지 진행

In [57]:
tr_input_df.columns

Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
       'x_10'],
      dtype='object')

In [61]:
pd.DataFrame(importance_res['importances_mean'].reshape(1,-1), columns=tr_input_df.columns )

# x_0의 중요도가 가장 낮음 

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
0,0.002326,0.025924,0.135379,0.004532,0.046509,0.034427,0.017805,0.03663,1.961254,0.178107,1.015712


In [65]:
random_rfr.fit(tr_input_df.drop('x_0', axis= 1).values, tr_output_df.values)

# oob 데이터를 검증용 데이터로 사용하기 떄문에 굳이 train_test_split 필요 없음
# x_0를 제거한 뒤 다시 RandomForest Regression + Random_Search 사용 

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, oob_score=True,
                                                   random_state=43),
                   n_iter=100,
                   param_distributions={'max_depth': [-1, 4, 5],
                                        'max_features': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'min_samples_leaf': [10, 11, 12],
                                        'n_estimators': range(10, 150, 20)},
                   scoring='neg_mean_squared_error')

In [66]:
random_rfr_best_model = random_rfr.best_estimator_
random_rfr_best_model.fit(tr_input_df.drop('x_0', axis= 1).values, tr_output_df.values)

random_rfr_best_model.oob_score_

0.6134744756241156

In [68]:
importance_res = permutation_importance(random_rfr_best_model,
                                        tr_input_df.drop('x_0', axis= 1).values,
                                        tr_output_df.values,
                                        n_repeats= 10,
                                        scoring='neg_mean_squared_error',
                                        random_state=1)

In [72]:
pd.DataFrame(importance_res['importances_mean'].reshape(1,-1), columns=tr_input_df.drop('x_0', axis= 1).columns)

# 2. x_6 제거 

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10
0,0.025321,0.134743,0.004482,0.038911,0.048579,0.016034,0.042695,1.322029,0.19783,1.38565


In [73]:
random_rfr.fit(tr_input_df.drop(['x_0', 'x_6'], axis= 1).values, tr_output_df.values)

# oob 데이터를 검증용 데이터로 사용하기 떄문에 굳이 train_test_split 필요 없음
# 또 특성이 가장 낮은 x_1을 제거 후 학습

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, oob_score=True,
                                                   random_state=43),
                   n_iter=100,
                   param_distributions={'max_depth': [-1, 4, 5],
                                        'max_features': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'min_samples_leaf': [10, 11, 12],
                                        'n_estimators': range(10, 150, 20)},
                   scoring='neg_mean_squared_error')

In [74]:
random_rfr_best_model = random_rfr.best_estimator_
random_rfr_best_model.fit(tr_input_df.drop(['x_0', 'x_6'], axis= 1).values, tr_output_df.values)

random_rfr_best_model.oob_score_

# 약간 상능한 정도라 의미 없음

# 위와 같은 방법으로 변수를 제거할 수도 있고 별 차이가 없다면 다른 방법을 찾아봐야함

0.6135730565567508

In [167]:
# 모델을 바꿔서 적용해보기

lgb_reg = lgb.LGBMRegressor()

lgbreg_random_params ={
            'max_depth' : [-1,3,4,5],  # 파라미터 지정하지 않으면 기본 값-1 부여 -> 이는 제한을 두지 않겠다는 의미
            'num_leaves' : [7],  # 트리 당 리프 개수  보통 2^max_depth 보다 1 작게!  (2^3-1 = 7)
            'n_estimators' : range(30,100,10),  # 트리 개수  보통 학습률 작으면 크게함, 너무 크면 과적합 위험 있음
            'random_state': [43], # 재현성 보장
            'min_child_samples': [20,21,22,23],  # 하위 노드 최소 데이터 개수 
            'n_jobs' : [-1], 
            'learning_rate': np.arange(0.05, 0.2, 0.05),  # 학습률
            'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # 전체 데이터셋에서 사용할 데이터 비율 (일반화 성능, 과적합 방지 위한 파라미터)
            'colsample_bytree' : [0.6, 0.7, 0.8, 0.9, 1.0]  # 사용할 피쳐의 비율
        }

n_iter = 200

random_lgb_reg = RandomizedSearchCV(lgb_reg,
                                    param_distributions = lgbreg_random_params,
                                    n_iter = n_iter,
                                    cv = 10,
                                    scoring= 'neg_mean_squared_error',
                                    )

# 추가로, feature_importance 구할 수 있는데, 'gain' , 'split' 두 가지 기준이 존재
# split은 트리 내 분기 시 역핧을 크게 한 변수에 중요도를 더욱 부여하는 것이고
# gain은 말 그래도 정보 이득. 즉, pre_split, post_split간 차이를 보고 기여를 많이 한 변수에 중요도를 더욱 부여하는 것 

In [168]:
lgb_obj = random_lgb_reg.fit(x_tr,y_tr)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [169]:
print(lgb_obj.best_score_)
print(lgb_obj.best_estimator_)
print(lgb_obj.best_params_)

-2.73056468623403
LGBMRegressor(max_depth=3, min_child_samples=23, n_estimators=90, n_jobs=-1,
              num_leaves=7, random_state=43, subsample=0.6)
{'subsample': 0.6, 'random_state': 43, 'num_leaves': 7, 'n_jobs': -1, 'n_estimators': 90, 'min_child_samples': 23, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 1.0}


In [170]:
lgb_best_model = lgb_obj.best_estimator_ 

# 최적의 파라미터 추출

In [171]:
lgb_best_model.fit(x_tr, y_tr, 
                    eval_set=[(x_val, y_val)],
                    eval_metric='mse')

# 최적의 파라미터로 다시 학습

LGBMRegressor(max_depth=3, min_child_samples=23, n_estimators=90, n_jobs=-1,
              num_leaves=7, random_state=43, subsample=0.6)

In [172]:
lgb_best_model.best_score_  

# mse 값 확인, 낮을 수록 좋음

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('l2', 3.039391883554995)])})

In [118]:
lgb_preds = lgb_best_model.predict(test_df.drop(['ID' ,'x_0', 'x_6'],axis = 1).values)

In [119]:
subm_df['y'] = lgb_preds 

# MSE 값이 가장 좋았을 때(낮았을 때)의 예측 값을 저장